In [ ]:
# выполнялось в Pycharm наи ПК, если использовать на колабе, необходимо сначала примонтировать гугл диск и загрузить в content датасет
# данный скрипт позволяет просматривать метки в датасете, листая их с помощью стралок, а также регулировать цвет и величину шрифта названий классов
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import os
import cv2
import numpy as np
import random

images_dir = r'C:\Users\Admin\Desktop\Project_land_use\13_new\train\images'
labels_dir = r'C:\Users\Admin\Desktop\Project_land_use\13_new\train\labels'
class_names = ['Agricultural object', 'Fields', 'Forest', 'Grassland', 'Power lines', 'Public facility', 'Residential facility', 'Road', 'Water']

class ImageBrowser:
    def __init__(self, images, labels, class_names):
        self.images = images
        self.labels = labels
        self.class_names = class_names
        self.index = 0
        self.fig, self.ax = plt.subplots()
        self.update()

        # Добавляем кнопки
        ax_prev = plt.axes([0.7, 0.05, 0.1, 0.075])
        ax_next = plt.axes([0.81, 0.05, 0.1, 0.075])
        self.b_next = Button(ax_next, 'Next')
        self.b_next.on_clicked(self.next)
        self.b_prev = Button(ax_prev, 'Previous')
        self.b_prev.on_clicked(self.prev)
        plt.show()

    def display_image_with_labels(self, image_path, label_path):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            class_name = self.class_names[class_id]
            coords = np.array(parts[1:], dtype=np.float32)
            coords = coords.reshape(-1, 2)

            h, w = image.shape[:2]
            coords[:, 0] *= w
            coords[:, 1] *= h
            coords = coords.astype(np.int32)

            cv2.polylines(image, [coords], isClosed=True, color=(0, 255, 0), thickness=2)

            # Вычисляем центроид полигона
            centroid = np.mean(coords, axis=0).astype(np.int32)

            # Добавляем название класса в центре полигона? синиЙ, покрупнее.
            cv2.putText(image, class_name, tuple(centroid), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)

        return image

    def update(self):
        image_path = self.images[self.index]
        label_path = self.labels[self.index]
        image = self.display_image_with_labels(image_path, label_path)
        self.ax.clear()
        self.ax.imshow(image)
        self.ax.set_title(f"Image {self.index + 1}/{len(self.images)}")
        plt.draw()

    def next(self, event):
        self.index = (self.index + 1) % len(self.images)
        self.update()

    def prev(self, event):
        self.index = (self.index - 1) % len(self.images)
        self.update()

image_files = [os.path.join(images_dir, f) for f in os.listdir(images_dir)]
label_files = [os.path.join(labels_dir, os.path.splitext(f)[0] + ".txt") for f in os.listdir(images_dir)]

browser = ImageBrowser(image_files, label_files, class_names)